In [ ]:
!sudo apt-get install libmagickwand-dev
!pip install --no-cache-dir \
    opencv-python-headless==4.6.*\
    rawpy==0.17.* \
    pandas \
    Pillow==7.1.2 \
    scikit-image==0.16.2 \
    scipy \
    tqdm \
    Wand

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libmagickwand-dev is already the newest version (8:6.9.10.23+dfsg-2.1ubuntu11.4).
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!git clone https://github.com/andreacos/BoostingCNN-Jpeg-Primary-Quantization-Matrix-Estimation

fatal: destination path 'BoostingCNN-Jpeg-Primary-Quantization-Matrix-Estimation' already exists and is not an empty directory.


In [ ]:
import sys

sys.path.insert(1, "BoostingCNN-Jpeg-Primary-Quantization-Matrix-Estimation")

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import io
import os
from glob import glob

import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import networks #import (custom_mse_wrapper, custom_softmax_activation,
                  #    custom_two_terms_loss_wrapper)
from PIL import Image
from skimage.util import view_as_windows
import utils #import label2coefficient
#, max_min_coefficient, string2Q

In [ ]:
model_file = 'BoostingCNN-Jpeg-Primary-Quantization-Matrix-Estimation/models/model_QF1_60-98_QF2_90-2-term-loss.h5'
image_file =  'BoostingCNN-Jpeg-Primary-Quantization-Matrix-Estimation/resources/00000000_redaf7d93t.TIF_85_90.png'
_map = np.load('BoostingCNN-Jpeg-Primary-Quantization-Matrix-Estimation/resources/qf1_qf2_map_90.npy', allow_pickle=True)

In [ ]:
def preprocess(image: np.array, dimension: tuple) -> np.array:
    if isinstance(image, str):
        image = cv2.imread(image)    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)[:,:,0]
    if image.shape != dimension:
        image = cv2.resize(image, dimension)
    return image.astype(np.float32) / 255.

#WMSE
def mse_weighted(pred: np.array, dct_coeffs: np.array, weights: np.array):
  diff = np.asarray((pred - dct_coeffs)**2) * weights
  weighted = np.average(diff, axis=1, weights=weights)
  return weighted

#MSE
def mse(pred: np.array, dct_coeffs: np.array):
    return np.mean((pred - dct_coeffs) ** 2, axis=1)


def make_predictWMSE(image: np.array, arr: np.array, maximum_coefficients: np.array, q50_coeffs: np.array) -> np.array:
    predicted_label = label2coefficient(model.predict(np.expand_dims(image, [0, -1]), verbose=0).flatten(), maximum_coefficients)
    return labels[np.argmin(mse_weighted(predicted_label, arr, q50_coeffs))]

def make_predictMSE(image: np.array, arr: np.array, maximum_coefficients: np.array) -> np.array:
    predicted_label = label2coefficient(model.predict(np.expand_dims(image, [0, -1]), verbose=0).flatten(), maximum_coefficients)
    return labels[np.argmin(mse(predicted_label, arr))]

def make_predict(image: np.array, arr: np.array, maximum_coefficients: np.array, q50_coeffs: np.array) -> np.array:
    w_mse_idx = make_predictWMSE(image, arr, maximum_coefficients, q50_coeffs)
    mse_idx = make_predictMSE(image, arr, maximum_coefficients)
    return w_mse_idx, mse_idx

def image_compression(image: np.array, *qfs: tuple) -> np.array:
    for qf in qfs:
        image = np.array(Image.open(io.BytesIO(cv2.imencode('.jpg', image, [int(cv2.IMWRITE_JPEG_QUALITY), qf])[1])))
    return image

In [ ]:
maximum_coefficients, _ = max_min_coefficient(
    quality_range=(50, 100),
    n_coeffs=15
    )

In [ ]:
model = tf.keras.models.load_model(
    model_file,
    custom_objects=({
        'custom_softmax': custom_softmax_activation(maximum_coefficients),
        'custom_two_terms_loss_wrapper': custom_two_terms_loss_wrapper(maximum_coefficients, 0.8),
        'custom_mse': custom_mse_wrapper(maximum_coefficients)
        })
    )

In [ ]:
label2coefficient(
    model.predict(np.expand_dims(preprocess(image_file, (64, 64)), [0, -1])).flatten(), 
    max_coefficients=maximum_coefficients
    )

1/1 [==============================] - 2s 2s/step


array([ 7,  5,  4,  6,  4,  3,  7,  6,  4,  6,  8,  7,  7,  8, 11])

In [ ]:
labels = _map[:, :2]
dct_coeffs = np.array([string2Q(i[2], size=(8, 8), flatten=True)[:15] for i in _map])
#Задание весов для WMSE
q50_coeffs = np.array([16,11,12,14,12,10,16,14,13,14,18,17,16,19,24], dtype='int64')

In [ ]:
examples = [
    (60, ),
    (65, 75),
    (70, 90),
    (75, 70),
    (95, 95)
]

### A0: Программно сгенерировать 5 принципиально различных ситуаций: однократное сжатие, 𝑄𝐹1 < 𝑄𝐹2, 𝑄𝐹1 ≪ 𝑄𝐹2, 𝑄𝐹1 > 𝑄𝐹2, 𝑄𝐹1 ≈ 𝑄𝐹2. 

In [ ]:
def a0_task(examples: list) -> pd.DataFrame:
    a0_data = []

    for example in examples:
        for path in glob("/content/gdrive/MyDrive/Colab Notebooks/images/*.tif"):
            image = image_compression(cv2.imread(path), *example)[:64, :64]
            pred_w, pred = make_predict(preprocess(image, (64, 64)), dct_coeffs, maximum_coefficients, q50_coeffs)
            a0_data.append([example[0], pred_w[0], np.abs(pred_w[0]-example[0]), pred[0], np.abs(pred[0]-example[0])])

    task_a0 = pd.DataFrame(a0_data, columns=['QF1', 'Prediction_WMSE', 'Error_WMSE', 'Prediction', 'Error'])
    return task_a0.groupby('QF1').mean().reset_index()

In [ ]:
a0_task(examples)

QF1  Prediction_WMSE  Error_WMSE  Prediction     Error
0   60        62.000000    2.000000   61.916667  1.916667
1   65        67.750000    3.750000   67.833333  3.833333
2   70        67.750000    3.083333   67.750000  3.083333
3   75        68.916667    6.083333   69.000000  6.000000
4   95        95.083333    1.916667   95.083333  1.750000

### A1: Сравнить ошибку при оценивании 𝑄𝐹1 по одному патчу и при оценивании по 𝑝 = 10 патчам, выбранным из одного изображения без пересечения.

In [ ]:
def a1_task(examples: list) -> pd.DataFrame:
    a1_data = []

    for example in examples:
        for path in glob("/content/gdrive/MyDrive/Colab Notebooks/images/*.tif"):
            chunks = view_as_windows(image_compression(cv2.imread(path), *example), (64, 64, 3), 64).reshape(-1, 64, 64, 3)
            temp_datawmse = []
            temp_datamse = []
            
            for chunk in chunks[:10]:
                pred_w, pred = make_predict(preprocess(chunk, (64, 64)), dct_coeffs, maximum_coefficients, q50_coeffs)
                temp_datawmse.append([pred_w[0], np.abs(pred_w[0] - example[0])])
                temp_datamse.append([pred[0], np.abs(pred[0] - example[0])])
        
            mean_resultwmse = np.mean(temp_datawmse, axis=0)
            mean_resultmse = np.mean(temp_datamse, axis=0)
            a1_data.append([example[0], mean_resultwmse[0], mean_resultwmse[1], mean_resultmse[0], mean_resultmse[1]])
    task_a1 = pd.DataFrame(a1_data, columns=['QF1', 'Prediction_WMSE', 'Error_WMSE', 'Prediction', 'Error'])
    return task_a1.groupby('QF1').mean().reset_index()

In [ ]:
a1_task(examples)

QF1  Prediction_WMSE  Error_WMSE  Prediction     Error
0   60        61.808333    1.808333   61.800000  1.800000
1   65        67.250000    3.533333   67.250000  3.483333
2   70        68.075000    2.541667   68.083333  2.533333
3   75        69.158333    5.841667   69.175000  5.825000
4   95        94.908333    2.958333   94.850000  2.866667